# Bài tập
Dùng keras để xây dựng mô hình deep learning để nhận dạng logo của các hãng xe.<br>
Có 8 loại logo cần nhận dạng như sau:<br>
<img src="logo_classes.png" alt="Classes" style="width:224px; heoght:224px"/><br>
Ảnh ví dụ để test mô hình, kích thước của ảnh test sẽ được scale về <strong>224x224</strong>:<br>
<img src="Lexus.jpg" alt="Lexus" style="width:224px; heoght:224px"/><br>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import ResNet50, VGG19, VGG16, ResNet101V2
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, load_model
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, accuracy_score
from tensorflow.keras.layers import Conv2D, Dropout, Dense, Flatten, MaxPooling2D, GlobalAveragePooling2D, Activation, BatchNormalization, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2

In [3]:
def load_images(folder, class_map):
    images = []
    labels = []
    for brand, index in class_map.items():
        brand_path = os.path.join(folder, brand)
        if os.path.isdir(brand_path):
            for img_file in os.listdir(brand_path):
                img_path = os.path.join(brand_path, img_file)
                img = cv2.imread(img_path)
                img = cv2.resize(img, (224, 224))
                img = img / 255.0
                images.append(img)
                labels.append(index)
    return np.array(images), np.array(labels)

In [10]:
class_map = {brand: idx for idx, brand in enumerate(os.listdir("/content/drive/MyDrive/DeepLearning/Train"))}

In [11]:
X_train, y_train = load_images("/content/drive/MyDrive/DeepLearning/Train", class_map)
X_test, y_test = load_images("/content/drive/MyDrive/DeepLearning/Test", class_map)

In [12]:
y_train = to_categorical(y_train, 8)
y_test = to_categorical(y_test, 8)

In [13]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.95,
    zoom_range=0.95,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/DeepLearning/Train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset="training")

validation_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/DeepLearning/Train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset="validation")

Found 2013 images belonging to 8 classes.
Found 500 images belonging to 8 classes.


In [14]:
base_model = ResNet101V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers[:-50]:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu', kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = Dropout(0.6)(x)
out = Dense(8, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=out)

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

171317808/171317808 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [15]:
early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.001, patience=15, restore_best_weights=True)
checkpoint = ModelCheckpoint("best_model_cnn.keras", monitor='val_accuracy', save_best_only=True, verbose=1)
history = model.fit(
        train_generator,
        epochs=5,
        validation_data=validation_generator,
        batch_size = 64,
        callbacks = [early_stopping, checkpoint])

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 873ms/step - accuracy: 0.3940 - loss: 9.4069
Epoch 1: val_accuracy improved from -inf to 0.15800, saving model to best_model_cnn.keras
63/63 ━━━━━━━━━━━━━━━━━━━━ 123s 1s/step - accuracy: 0.3954 - loss: 9.3870 - val_accuracy: 0.1580 - val_loss: 255.3788
Epoch 2/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 647ms/step - accuracy: 0.6356 - loss: 5.6263
Epoch 2: val_accuracy improved from 0.15800 to 0.39600, saving model to best_model_cnn.keras
63/63 ━━━━━━━━━━━━━━━━━━━━ 97s 922ms/step - accuracy: 0.6361 - loss: 5.6176 - val_accuracy: 0.3960 - val_loss: 14.2152
Epoch 3/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 626ms/step - accuracy: 0.7332 - loss: 3.7081
Epoch 3: val_accuracy improved from 0.39600 to 0.72000, saving model to best_model_cnn.keras
63/63 ━━━━━━━━━━━━━━━━━━━━ 87s 995ms/step - accuracy: 0.7331 - loss: 3.7035 - val_accuracy: 0.7200 - val_loss: 3.4244
Epoch 4/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 640ms/step - accuracy: 0.7516 - loss: 2.6498
Epoch 4: val_accuracy improve

In [17]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

test_data = test_datagen.flow_from_directory(
    "/content/drive/MyDrive/DeepLearning/Test",
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    shuffle=False
)

Found 400 images belonging to 8 classes.


In [18]:
loss, accuracy = model.evaluate(test_data)
print(f"Test Accuracy: {accuracy:.4f}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.8455 - loss: 2.7469
Test Accuracy: 0.8375
